In [1]:
%pip install --quiet fastapi uvicorn nest-asyncio

Note: you may need to restart the kernel to use updated packages.


> why is nest_asyncio required?

#### ChatGPT

The `nest_asyncio` library is used to "patch" Python's standard library asyncio event loop to allow nested use of `asyncio.run` and event loops. This is particularly useful in environments like Jupyter Notebook where the event loop is already running in the background to handle things like cell execution and various I/O operations.

In a standard Python script, you generally control the event loop yourself. When you execute an ASGI app using Uvicorn or any other ASGI server, the server takes control of the event loop to handle HTTP requests asynchronously.

In contrast, Jupyter Notebook already has an event loop running to manage its interactive environment. This can lead to issues when you try to run another event loop for your FastAPI application on top of it. By using `nest_asyncio`, you patch the existing event loop to support nesting, effectively allowing the FastAPI event loop to run inside the Jupyter event loop without conflicts.

Without `nest_asyncio`, you'd likely run into an error like:

```
RuntimeError: This event loop is already running
```

This library resolves that issue, enabling you to run asynchronous code like FastAPI apps directly within Jupyter cells. Given your technical background, you can think of `nest_asyncio` as a workaround to make two event-driven systems coexist peacefully within the same Python process.

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
%cd G:\ml\glados-tts-upstream

G:\ml\glados-tts-upstream


In [4]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.
torchaudio 0.13.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.


In [5]:
# https://github.com/R2D2FISH/glados-tts
from glados import tts_runner
glados = tts_runner(False, True)

Initializing TTS Engine...


In [6]:
glados.speak("Hello, world!", True)

Forward Tacotron took 75.19173622131348ms
HiFiGAN took 540.1718616485596ms


In [7]:
from fastapi import FastAPI, Request
app = FastAPI()

In [8]:
@app.get("/")
def read_root():
    return {"message": "Hello, World!"}

In [9]:
@app.post("/say")
async def echo(request: Request):
    body = await request.body()
    text = body.decode("utf-8")
    glados.speak(text, True)
    return {"echo": text}


In [10]:
def run_glados_server():
    import uvicorn
    # The host and port parameters are optional
    uvicorn.run(app, host="0.0.0.0", port=8000)
    # this gets weird because the output gets dumped in future cells, whatever

import threading
glados_thread = threading.Thread(target=run_glados_server)
glados_thread.start()

In [11]:
import requests
requests.post("http://localhost:8000/say", data="Hello, world!")

INFO:     Started server process [26644]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Forward Tacotron took 73.23813438415527ms
HiFiGAN took 539.7112369537354ms
INFO:     127.0.0.1:54105 - "POST /say HTTP/1.1" 200 OK


<Response [200]>

In [12]:
print("hi")

hi


In [13]:
%pip install --quiet SpeechRecognition openai-whisper loguru

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.16.0%2Bcu118-cp310-cp310-win_amd64.whl (5.0 MB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.1.0%2Bcu118-cp310-cp310-win_amd64.whl (2722.7 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.1.0%2Bcu118-cp310-cp310-win_amd64.whl (3.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\TeamD\\.conda\\envs\\torch\\Lib\\site-packages\\~-rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [18]:
import torch
torch.cuda.is_available()

False

In [16]:
import threading
import speech_recognition as sr
import requests
import numpy as np
import whisper
from loguru import logger

def continuous_transcription_whisper():
    try:
        r = sr.Recognizer()
        audio_model = whisper.load_model("large-v2", device="cuda")
        while True:  # Add an exit condition if needed
            with sr.Microphone(sample_rate=16000) as source:
                logger.info("Listening...")
                audio = r.listen(source)
                np_audio = np.frombuffer(audio.get_raw_data(), np.int16).flatten().astype(np.float32) / 32768.0

                try:
                    result = audio_model.transcribe(np_audio, batch_size=16)
                    text = str(result["text"]).strip()  # Extract the transcribed text
                    logger.info(f"Transcribed: {text}")
                    
                    # Sending the transcribed text to your FastAPI server
                    requests.post("http://localhost:8000/say", data=text)
                    
                except Exception as e:  # Modify this to catch specific exceptions you're expecting
                    logger.error(f"Error in transcription: {e}")
    except Exception as e:
        logger.error(f"Error in transcription background task: {e}")

transcription_thread = threading.Thread(target=continuous_transcription_whisper)
transcription_thread.start()

In [17]:
print("hi")

hi


2023-10-22 01:15:57.453 | ERROR    | __main__:continuous_transcription_whisper:29 - Error in transcription background task: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.
